In [1]:
import pandas as pd


#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
#spacy
import spacy

from nltk.corpus import stopwords
#vis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
stopwords = stopwords.words('english')

df = pd.read_csv('C:/Users/LEVEL51PC/Desktop/Essex/Frontrunner/premierleague/cleaned_data.csv',index_col=0)
df.dropna(inplace=True)
titles = df['title'].tolist()

def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

lemmatized_titles = lemmatization(titles)

data_words = gen_words(lemmatized_titles)

id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)





In [3]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

C:\Users\LEVEL51PC\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.215686  0.364713       1        1  12.533766
2      0.113511 -0.406909       2        1  12.187370
7      0.064835  0.397269       3        1  12.062897
6     -0.216982 -0.370565       4        1  10.869913
5     -0.355343  0.135838       5        1   9.946807
8      0.279314  0.267034       6        1   9.653799
3      0.328973 -0.214550       7        1   9.082663
4     -0.371698 -0.117296       8        1   8.308283
0      0.399051  0.038672       9        1   7.812110
9     -0.025975 -0.094208      10        1   7.542391, topic_info=         Term        Freq       Total Category  logprob  loglift
35      match  920.000000  920.000000  Default  10.0000  10.0000
620    second  763.000000  763.000000  Default   9.0000   9.0000
20     target  918.000000  918.000000  Default   8.0000   8.0000
1209  preview  739.000000  739.000000  Default   7.0000   7.0000
18      watch  458.000000  458.000000  Default   6.0000   6.0000
...       ...         ...         ...      ...      ...      ...
553       put   94.938358   95.939613  Topic10  -3.1021   2.5741
212      lead   85.207330   86.208533  Topic10  -3.2103   2.5729
590    safety   80.451613   81.453193  Topic10  -3.2677   2.5723
584      play   48.464573   49.465769  Topic10  -3.7745   2.5642
55      first   48.442587   49.443770  Topic10  -3.7750   2.5642

[110 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
113       1  0.989058      again
167       6  0.989799        aim
89        5  0.986371    appoint
184      10  0.992180    arsenal
194       8  0.992891       back
...     ...       ...        ...
463       5  0.991376  watchlist
830       1  0.990940        way
147       1  0.995411        win
870       7  0.984391     winner
63       10  0.991494       wolf

[100 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 8, 7, 6, 9, 4, 5, 1, 10])

In [5]:
coherence_model_lda = CoherenceModel(model=lda_model,texts=data_words,dictionary=id2word,coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print("Coherence Score:",coherence_lda)

Coherence Score: 0.5920028175228378


In [2]:
# Find the number of topic
model_coherence = []
for num_topic in range(5,21):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topic,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")
    coherence_model_lda = CoherenceModel(model=lda_model,texts=data_words,dictionary=id2word,coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    model_coherence.append((num_topic,coherence_lda))
    print("Number of topic, Coherence Score:",num_topic,coherence_lda)

Number of topic, Coherence Score: 5 0.5773474504991962
Number of topic, Coherence Score: 6 0.5535846347896409
Number of topic, Coherence Score: 7 0.5648594624562703
Number of topic, Coherence Score: 8 0.5815068324556083
Number of topic, Coherence Score: 9 0.5929783854870642
Number of topic, Coherence Score: 10 0.5920028175228378
Number of topic, Coherence Score: 11 0.5984784329640817
Number of topic, Coherence Score: 12 0.5791060353392495
Number of topic, Coherence Score: 13 0.5756265448687741
Number of topic, Coherence Score: 14 0.5673753348824632
Number of topic, Coherence Score: 15 0.5552402067560586
Number of topic, Coherence Score: 16 0.5476874483573031
Number of topic, Coherence Score: 17 0.5657882569034444
Number of topic, Coherence Score: 18 0.570652521680472
Number of topic, Coherence Score: 19 0.5585318366366931
Number of topic, Coherence Score: 20 0.5599337307574468
